<a href="https://colab.research.google.com/github/iamabd53/Resume-Parser-NER-Project/blob/main/Resume_Parser_NER_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing and importing necessary libraries
!pip install -U spacy
!pip install spacy_transformers
!pip install PyMuPDF
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 7.4 MB/s eta 0:00:00


In [ ]:
# Using tesla T4 GPU
!nvidia-smi

Sun Dec 24 05:44:09 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Loading the data
data_cv = json.load(open('/content/drive/MyDrive/ResumeParser/dataset/dataset.json','r'))

In [ ]:
# length of data
len(data_cv)

1014

In [ ]:
data_cv[0]

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [ ]:
# config file of spaCy -> Component - Name Entity Recognition , Hardware - GPU (Transformer)
# filling the remaning default in our base_config file
!python -m spacy init fill-config /content/drive/MyDrive/ResumeParser/config/base_config.cfg /content/drive/MyDrive/ResumeParser/config/config.cfg

2023-12-24 05:46:54.058729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 05:46:54.058790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 05:46:54.060093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-24 05:46:55.107394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/ResumeParser/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
# This function "get_spacy_doc" processes text and entity annotations, creates spaCy documents with those entities,
# and stores them in a DocBin object for later use in training a spaCy NLP model
def get_spacy_doc(file, data):
    # Loading spacy
    nlp = spacy.blank('en')
    # Creating Docbin
    db = DocBin()

    # Iterating through data
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        # Initializes empty lists to store entities (ents) and indices of processed entities (entity_indices).
        ents = []
        entity_indices = []

        # Iterates through each entity in the annotations
        for start, end, label in annot:
            skip_entity = False # Checks for overlapping entities by iterating through character indices of the current entity. If an index is already in entity_indices, it means there's an overlap, and the entity is skipped
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue
            # Marks the character indices of the current entity as processed.
            entity_indices = entity_indices + list(range(start, end))

            # Attempts to create a spaCy Span object for the entity. If there's an exception (e.g., invalid label), it skips the entity.
            try:
                span = doc.char_span(start, end,label=label,alignment_mode = 'strict')
            except:
                continue

            # checks whether the span object is None, which would indicate that the creation was not successful.
            if span is None:
                err_data = str([start, end]) + " " + str(text) + "\n "
                file.write(err_data)
            # Appends the successfully created Span object to the ents list.
            else:
                ents.append(span)

        try:
            # Assigning entities and adding to DocBin
            doc.ents = ents
            db.add(doc)
        except:
            pass
    # Returning DocBin
    return db

In [ ]:
# splitting the data
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_cv, test_size=0.3)

In [ ]:
# Saving train and test data inside the model folder
file = open('/content/drive/MyDrive/ResumeParser/model/train_file.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/ResumeParser/model/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/ResumeParser/model/test_data.spacy')

file.close()

100%|██████████| 305/305 [00:02<00:00, 122.74it/s]


In [ ]:
# Training             # part of config file                                   # save the model output inside the model folder           # path of train data                                                     # path of test data                                                  # using gpu to train the model
!python -m spacy train /content/drive/MyDrive/ResumeParser/config/config.cfg --output /content/drive/MyDrive/ResumeParser/model/output --paths.train /content/drive/MyDrive/ResumeParser/model/train_data.spacy --paths.dev /content/drive/MyDrive/ResumeParser/model/test_data.spacy --gpu-id 0

2023-12-24 05:53:20.641743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 05:53:20.641791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 05:53:20.643017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-24 05:53:21.674883: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory:
/content/drive/MyDrive/ResumeParser/model/output
ℹ Saving to output directory:
/content/drive/MyDrive/ResumeParser/model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===================

* Using transformer
* Total step is 7000

In [ ]:
# loading the model
resume_parser = spacy.load('/content/drive/MyDrive/ResumeParser/model/output/model-best')

In [ ]:
# Loading the sample resume
import fitz

sample_resume = '/content/drive/MyDrive/ResumeParser/test/sample_resume.pdf'
doc = fitz.open(sample_resume)


In [ ]:
# Extracting text from sample resume
text = ' '
for page in doc:
  text = text + str(page.get_text())

In [ ]:
# text display
text

" Vijay Pagare\n(+91)889XXXXX28 | xyz@gmail.com | Miraroad, Thane, MH, IND https://\nwww.linkedin.com/in/xyz/\nA frontend-leaning software engineer who has 4.5+ years of experience in building and maintaining high-quality (B2B)\nsaas products and web applications. Proven ability to work independently and as part of a team in fast-moving,\nresource-constraint environments where short turnaround times are a norm. Exceptional at leveraging interpersonal skills\nto facilitate a collaborative relationship among cross-functional teams to get the work done. Excellent problem-solver with\nan aptitude for troubleshooting and the ability to quickly master new skills, technology, or a role.\nPROFESSIONAL EXPERIENCE\nPROPELLOR.AI\nSoftware Engineer - Frontend\nPune - Remote\nAugust 2021–Present\n●\nArchitected, built and maintained business critical modules for a data uni��cation and visualisation\nplatform.\n○\nIntroduced 20+ charts including sankey, wordcloud, heatmap, tree, bubble, Map - India 

In [ ]:
# Process the text with resume_parser model
doc = resume_parser(text)

# Collect the entities and their labels in a list of dictionaries
entities = []
for ent in doc.ents:
    entities.append({"text": ent.text, "label": ent.label_})

# Create the JSON output
json_output = json.dumps(entities, indent=4)  # Indent for readability

# Print or save the JSON output
print(json_output)


[
    {
        "text": "www.linkedin.com/in/xyz/\nA frontend-leaning software engineer who has 4.5+ years of experience in building and maintaining high-quality (B2B)\nsaas products and web applications. Proven ability to work independently and as part of a team in fast-moving,\nresource-constraint environments where short turnaround times are a norm. Exceptional at leveraging interpersonal skills\nto facilitate a collaborative relationship among cross-functional teams to get the work done. Excellent problem-solver with\nan aptitude for troubleshooting and the ability to quickly master new skills, technology, or a role.\n",
        "label": "LINKEDIN LINK"
    },
    {
        "text": "PROFESSIONAL EXPERIENCE",
        "label": "COMPANIES WORKED AT"
    },
    {
        "text": "PROPELLOR.AI\n",
        "label": "COMPANIES WORKED AT"
    },
    {
        "text": "Software Engineer - Frontend",
        "label": "WORKED AS"
    },
    {
        "text": "Pune - Remote",
        "label": 